In [76]:
from config.constants import MONGODB_CREDENTIALS, SQL_LEAGUES_CONN, SQL_LEAGUES_ENGINE, API_KEY
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from riotwatcher import RiotWatcher

# MongoDB

In [2]:
mongodb = MongoClient(MONGODB_CREDENTIALS).slds

In [3]:
soloq_m_coll = mongodb.soloq_m
soloq_tl_coll = mongodb.soloq_tl

In [28]:
slo_tl_coll = mongodb.slo_tl

In [ ]:
slo_tl_coll.find_one({'gameId':})

In [4]:
sql = pymysql.connect(**SQL_LEAGUES_CONN)

In [5]:
cursor1 = sql.cursor()
query = 'SELECT game_id, realm, hash FROM slo'
cursor1.execute(query)
new_game_ids = [(gid[0], gid[1], gid[2]) for gid in cursor1]
platforms = [gid[1] for gid in new_game_ids]

In [6]:
set(platforms)

{'ESPORTSTMNT01', 'ESPORTSTMNT05', 'EUW1'}

In [7]:
cursor = soloq_m_coll.find({'platformId': {'$in': list(set(platforms))}}, {'_id': 0, 'gameId': 1, 'platformId': 1})

In [8]:
current_game_ids = [(gid['gameId'], gid['platformId']) for gid in cursor]

In [12]:
df = pd.DataFrame(current_game_ids)

In [49]:
dict1

{'gameCreation': {'$gteeee': 'meirae'}}

In [31]:
gs = soloq_m_coll.find({'gameCreation': {'$gte': 1526335200000, '$lte': 1526507999000}}, {'_id': 0, 'gameId': 1})

In [32]:
gids = [obj['gameId'] for obj in gs]

In [33]:
len(gids)

356

### Regex in MongoDB with game versions

In [193]:
patch = '8.1'

In [194]:
rgex_patch = '^' + '{}'.format(patch).replace('.', r'\.')

In [195]:
gamevers = [game['gameVersion'] for game in soloq_m_coll.find({'platformId': region, 'gameVersion': {'$regex': rgex_patch}}, {'_id': 0, 'gameVersion': 1})]

In [196]:
pd.Series(gamevers).value_counts()

8.1.214.5847    14
8.1.213.4336    13
dtype: int64

In [ ]:
g = soloq_m_coll.find({'participantIdentities.player.accountId': {'$in': [23787352]}}, {'_id': 0, 'gameId': 1})

In [ ]:
gs = [obj['gameId'] for obj in g]

In [ ]:
pd.DataFrame(gs).rename(columns={0: 'game_id'})

# SQL

In [19]:
import pymysql

In [84]:
engine = create_engine(SQL_LEAGUES_ENGINE)
cnx = engine.connect()
sql_df = pd.read_sql_table(con=cnx, table_name='slo', index_col='index')
cnx.close()

In [85]:
sql_df.head(3)

,season,split,date,series_id,week,event,game,game_id,realm,hash,...,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
index,,,,,,,,,,,,,,,,,,,,,
0,8,summer,26/05/2018 12:00:00,DRG_KIYF,1,BCN viewing party,1,750114,ESPORTSTMNT05,240839e4082e6796,...,Sacre,Hero,Vrow,Barcode,Homi,Orome,Aesthetic,Zazee,Carzzy,Rufus
1,8,summer,26/05/2018 13:00:00,DRG_KIYF,1,BCN viewing party,2,740189,ESPORTSTMNT05,bc2bed412a9b6474,...,Orome,Aesthetic,Zazee,Carzzy,Rufus,Sacre,Hero,Vrow,Barcode,Homi
2,8,summer,26/05/2018 14:00:00,EMZ_PGM,1,BCN viewing party,1,750170,ESPORTSTMNT05,4d9934aea20585ef,...,Moryo,Dahvys,Czekolad,Monk,Tarky,Glebo,Ippon,Siler,Javaaa,Karakal


In [79]:
competitions = sql_df[['competition', 'competition_abbv']]

In [82]:
teams = sql_df[['team_name', 'team_abbv']]

In [83]:
players = sql_df[['region', 'main_role', 'substitute', 'player_name', 'account_id', 'id', 'account_type']]

# Random

In [90]:
from riotwatcher import RiotWatcher
from config.constants import API_KEY
rw = RiotWatcher(API_KEY)

In [26]:
summ = rw.summoner.by_name(region='EUW1', summoner_name='Missing Q')

In [28]:
accid = summ['accountId']

In [30]:
ml = rw.match.matchlist_by_account(account_id=accid, region='EUW1')

In [35]:
gid = ml['matches'][0]['gameId']

In [36]:
gid

3669456176

In [44]:
tl = rw.match.timeline_by_match(match_id=3668417003, region='EUW1')

In [47]:
from itertools import chain

In [49]:
events = list(chain.from_iterable([frame['events'] for frame in tl['frames']]))

In [56]:
ward_df = pd.DataFrame([ward for ward in events if ward['type'] == 'WARD_PLACED'])

In [75]:
positions = {1: 'JUNG', 2: 'SUPP', 3: 'MID', 4: 'TOP', 5: 'ADC', 6: 'TOP', 7: 'JUNG', 8: 'MID', 9: 'ADC', 10:'SUPP'}

In [82]:
df_pos = pd.DataFrame(positions, index=(0,)).T.reset_index().rename(columns={'index': 'pid'})

In [85]:
ward_df2 = ward_df.merge(df_pos, left_on='creatorId', right_on='pid')

In [86]:
ward_df2.sort_values(['creatorId', 'timestamp']).to_excel('ward_test.xlsx')

In [87]:
ward_df2

,creatorId,timestamp,type,wardType,pid,0
0,7,70607,WARD_PLACED,YELLOW_TRINKET,7,JUNG
1,7,274523,WARD_PLACED,CONTROL_WARD,7,JUNG
2,7,302919,WARD_PLACED,YELLOW_TRINKET,7,JUNG
3,7,524776,WARD_PLACED,YELLOW_TRINKET,7,JUNG
4,7,744066,WARD_PLACED,YELLOW_TRINKET,7,JUNG
5,7,1181498,WARD_PLACED,CONTROL_WARD,7,JUNG
6,7,1366843,WARD_PLACED,CONTROL_WARD,7,JUNG
7,4,72059,WARD_PLACED,YELLOW_TRINKET,4,TOP
8,4,283698,WARD_PLACED,YELLOW_TRINKET,4,TOP
9,4,515090,WARD_PLACED,YELLOW_TRINKET,4,TOP
